In [31]:
import sympy as sp
import numpy as np
from functools import total_ordering
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Math, Latex, Markdown

InteractiveShell.ast_node_interactivity = "all"

## Поиск собственных значений и векторов

In [46]:
@total_ordering
class Eigenvector:
    def __init__(self, eigenvalue, alpha, eigenspace):
        self.eigenvalue = eigenvalue
        self.alpha = alpha
        self.eigenspace = eigenspace
        self.gamma = len(eigenspace)
        
    def __str__(self):
        return f"""
               eigenvalue = {self.eigenvalue},
               alpha = {self.alpha},
               gamma = {self.gamma}
               eigenspace = {str(self.eigenspace)}  
               """
    
    def __lt__(self, other):
        return self.eigenvalue < other.eigenvalue
    
    def __eq__(self, other): 
        if not isinstance(other, Eigenvector):
            return NotImplemented

        return self.eigenvalue == other.eigenvalue and self.alpha == other.alpha and self.gamma == other.gamma


def find_eigenvectors(M: sp.Matrix, name: str = "M") -> list:
    display(Markdown("## Посчитаем собственные вектора матрицы"))

    display(Markdown("### Сначала найдем характеристический многочлен"))
    lamb = sp.Symbol(r"\lambda")
    M_char_matrix = sp.Determinant(M - lamb * sp.eye(M.shape[0]))
    display(Math(name + " - " + lamb.name + r" \cdot E = "))
    display(M_char_matrix)
    M_char = M_char_matrix.simplify()
    display(M_char)

    display(Markdown("### Найдем его корни и их алгебраические кратности"))

    M_eigenvectors = M.eigenvects()

    for i in range(len(M_eigenvectors)):
        eigenvalue, alpha, _ = M_eigenvectors[i]
        display(Math(
            lamb.name + "_" + str(i + 1) + " = " + str(eigenvalue) + ", " + r"\alpha(" + str(eigenvalue) + ") = " + str(
                alpha)))

    display(Markdown("### Для каждого собственного значения найдем собственное подпространство"))

    M_result_eigenvectors = []
    for i in range(len(M_eigenvectors)):
        eigenvalue, alpha, eigenspace = M_eigenvectors[i]
        display(Math(lamb.name + "_" + str(i + 1) + " = " + str(eigenvalue) + ":"))

        M_eigen_system = M - eigenvalue * sp.eye(M.shape[0])
        display(M_eigen_system)

        display(Markdown("Перейдем к улучшенному ступенчатому виду"))
        M_eigen_system_rref, _ = M_eigen_system.rref()
        display(M_eigen_system_rref)

        display(Markdown("Т.о. собственное подпространство состоит из векторов: "))
        for eigenvector in eigenspace:
            display(eigenvector)

        gamma = len(eigenspace)
        display(Math(r"\gamma(" + str(eigenvalue) + ") = " + str(gamma)))

        M_result_eigenvectors.append(Eigenvector(eigenvalue, alpha, eigenspace))

    return M_result_eigenvectors


find_eigenvectors(sp.Matrix([
    [2, -1, -2],
    [-1, 5, 1],
    [-2, 1, 2],
]), "Q")

## Посчитаем собственные вектора матрицы

### Сначала найдем характеристический многочлен

<IPython.core.display.Math object>

Determinant(Matrix([
[2 - \lambda,          -1,          -2],
[         -1, 5 - \lambda,           1],
[         -2,           1, 2 - \lambda]]))

-\lambda**3 + 9*\lambda**2 - 18*\lambda

...

Перейдем к улучшенному ступенчатому виду

Matrix([
[1, 0,  1],
[0, 1, -2],
[0, 0,  0]])

Т.о. собственное подпространство состоит из векторов: 

Matrix([
[-1],
[ 2],
[ 1]])

<IPython.core.display.Math object>

## Ортогонализация Грама-Шмидта

In [16]:
def normalize(vector: sp.Matrix) -> sp.Expr:
    norm = sp.sqrt(vector.transpose().dot(vector))
    vector_normalized = vector / norm
    display(vector_normalized)
    return vector_normalized

## Диагонализация она же приведение квадратичной формы к каноническому виду (главным осям), она же приведение симметрической матрицы к каноническому виду

In [19]:
def diagonalize_with_normalization(M: sp.Matrix, name: str = "M") -> (sp.Matrix or None, sp.Matrix or None, list):
    M_eigenvectors = find_eigenvectors(M, name)

    for eigenvector in M_eigenvectors:
        if eigenvector.alpha != eigenvector.gamma:
            display(Markdown("Матрица не диагонализуема, так как"))
            display(
                Math(r"\alpha(" + str(eigenvector.eigenvalue) + r") \neq \gamma(" + str(eigenvector.eigenvalue) + ")"))
            return None, None, M_eigenvectors

    display(
        Markdown("Матрица диагонализуема, так как алгебраические кратности собственных векторов равны геометрическим"))

    D = np.zeros(M.shape)
    C = [0] * M.shape[0]
    position = 0
    for eigenvector in M_eigenvectors:
        display(Markdown("Нормализуем вектор собственного значения " + str(eigenvector.eigenvalue)))
        for i in range(eigenvector.alpha):
            D[position][position] = eigenvector.eigenvalue
            C[position] = np.array(normalize(eigenvector.eigenspace[i].transpose()))[0]
            position += 1

    D = sp.Matrix(D)
    C = np.array(C)
    C = sp.Matrix(C.T)

    display(Markdown("Диагональный вид матрицы " + name))
    display(D)
    display(Markdown("Матрица перехода C в базис, в котором достигается диагональный вид (hint D = C^-1 * A * C)"))
    display(C)

    return D, C, M_eigenvectors

def diagonalize(M: sp.Matrix, name: str = "M") -> (sp.Matrix or None, sp.Matrix or None, list):
    M_eigenvectors = find_eigenvectors(M, name)

    for eigenvector in M_eigenvectors:
        if eigenvector.alpha != eigenvector.gamma:
            display(Markdown("Матрица не диагонализуема, так как"))
            display(
                Math(r"\alpha(" + str(eigenvector.eigenvalue) + r") \neq \gamma(" + str(eigenvector.eigenvalue) + ")"))
            return None, None, M_eigenvectors

    display(
        Markdown("Матрица диагонализуема, так как алгебраические кратности собственных векторов равны геометрическим"))

    D = np.zeros(M.shape)
    C = [0] * M.shape[0]
    position = 0
    for eigenvector in M_eigenvectors:
        for i in range(eigenvector.alpha):
            D[position][position] = eigenvector.eigenvalue
            C[position] = np.array(eigenvector.eigenspace[i].transpose())[0]
            position += 1

    D = sp.Matrix(D)
    C = np.array(C)
    C = sp.Matrix(C.T)

    display(Markdown("Диагональный вид матрицы " + name))
    display(D)
    display(Markdown("Матрица перехода C в базис, в котором достигается диагональный вид (hint D = C^-1 * A * C)"))
    display(C)

    return D, C, M_eigenvectors


diagonalize(sp.Matrix([
    [2, -1, -2],
    [-1, 5, 1],
    [-2, 1, 2],
]), "Q")

## Посчитаем собственные вектора матрицы

### Сначала найдем характеристический многочлен

<IPython.core.display.Math object>

Determinant(Matrix([
[2 - \lambda,          -1,          -2],
[         -1, 5 - \lambda,           1],
[         -2,           1, 2 - \lambda]]))

-\lambda**3 + 9*\lambda**2 - 18*\lambda

...

Диагональный вид матрицы Q

Matrix([
[6.0,   0, 0],
[  0, 3.0, 0],
[  0,   0, 0]])

Матрица перехода C в базис, в котором достигается диагональный вид (hint D = C^-1 * A * C)

Matrix([
[-1, -1, 1],
[ 2, -1, 0],
[ 1,  1, 1]])

(Matrix([
 [6.0,   0, 0],
 [  0, 3.0, 0],
 [  0,   0, 0]]),
 Matrix([
 [-1, -1, 1],
 [ 2, -1, 0],
 [ 1,  1, 1]]),
  <__main__.Eigenvector at 0x105637190>])

## Приведение квадрики к каноническому виду

In [ ]:
def quadric_to_canonical(M: sp.Matrix, A: sp.Matrix):
    pass

## Приведение ортогонального оператора 3x3 в канонический вид

## Сингулярное разложение

In [45]:
def svd(M: sp.Matrix, name: str = "M") -> (sp.Matrix, sp.Matrix, sp.Matrix):
    L = M * M.transpose()
    display(Math("L = " + name + r" \cdot " + name + "^t = "))
    display(L)
    _, V, L_eigenvectors = diagonalize_with_normalization(L, "L")
    display(Markdown("Она же и будет матрицей V сингулярного разложения"))
    
    print("---------------")
    
    R = M.transpose() * M
    display(Math("R = " + name + r"^t \cdot " + name + " = "))
    display(R)
    _, U, R_eigenvectors = diagonalize_with_normalization(R, "R")
    display(Markdown("Если ее транспонировать, она и будет матрицей U сингулярного разложения"))
    U = U.transpose()
    display(U)
    
    print("---------------")
    
    display(Markdown("Найдем сингулярные числа матрицы"))
    common_eigenvectors = np.concatenate([L_eigenvectors, R_eigenvectors])
    common_eigenvectors = np.unique(common_eigenvectors)
    
    singular_values = []
    for i in range(len(common_eigenvectors)):
        eigenvalue = common_eigenvectors[i].eigenvalue
        singular_value = sp.sqrt(eigenvalue)
        display(Math("\delta_" + str(i) + " = \sqrt(" + str(eigenvalue) + ") = " + str(singular_value.simplify())))
        singular_values.append(singular_value)
    singular_values.reverse()
    
    display(Markdown("Таким образом матрица Σ = "))
    S = np.zeros(M.shape)
    for i in range(len(singular_values)):
        S[i][i] = singular_values[i]
    S = sp.Matrix(S)
    display(S)
    
    return V, S, U
        

svd(sp.Matrix([
    [-2, 2, 0],
    [2, 0, 4],
    [3, -2, 2]
]))

<IPython.core.display.Math object>

Matrix([
[  8, -4, -10],
[ -4, 20,  14],
[-10, 14,  17]])

## Посчитаем собственные вектора матрицы

### Сначала найдем характеристический многочлен

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Таким образом матрица Σ = 

Matrix([
[6.0,   0, 0],
[  0, 3.0, 0],
[  0,   0, 0]])

(Matrix([
 [ 2/3, -2/3, -1/3],
 [-1/3, -2/3,  2/3],
 [ 2/3,  1/3,  2/3]]),
 Matrix([
 [6.0,   0, 0],
 [  0, 3.0, 0],
 [  0,   0, 0]]),
 Matrix([
 [-2/3, -2/3, 1/3],
 [-1/3,  2/3, 2/3],
 [ 2/3, -1/3, 2/3]]))

## Ранговое приближение

In [ ]:
def rank_approximation(M: sp.Matrix, rank, name: str = "M"):
    display(Markdown("Посчитаем сингулярное разложение"))
    
    V, S, U = svd(M, name)
    
    display(Markdown("Уберем из Σ все сингулярные значение i > " + rank))
    
    S_new = np.zeros(S.shape)
    

rank_approximation(sp.Matrix([
    [-2, 2, 0],
    [2, 0, 4],
    [3, -2, 2]
]), 1)

# Ортогонализация Грама-Шмидта